In [ ]:
# # Cargar los datos en un dataframe
# internal = pd.read_excel('data/Internal_Bank_Dataset.xlsx')

# # Remplazamos el valor asignado como nulo para que no afecta al EDA
# internal.replace(-99999, np.nan, inplace=True)

In [ ]:
# # Visualizamos 5 primeras y 5 últimas líneas del dataframe
# print(head_tail(internal, 5))
# print("="*100)

# # Resumen estadístico de los datos numéricos
# print(internal.describe())

In [ ]:
# # Función que muestra la estructura de cada variable
# for col in internal.columns.tolist():
#     describe_columna(internal, col)